<a href="https://colab.research.google.com/github/HaonanZhang101045166/GNG5125/blob/main/Group_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
import nltk
import re
from urllib.request import urlopen
from nltk.corpus import stopwords
import random
import heapq
import numpy as np
import pandas as pd
import collections 



nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
book1="https://www.gutenberg.org/files/832/832-0.txt"
book2="https://www.gutenberg.org/cache/epub/10148/pg10148.txt"
book3="https://www.gutenberg.org/files/2892/2892-0.txt"
book4="https://www.gutenberg.org/cache/epub/10662/pg10662.txt"
book5="https://www.gutenberg.org/files/1251/1251-0.txt"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [174]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text


In [175]:
def data_partition(Url,label):
  raw=urlopen(Url).read()
  raw=raw.decode()
  partition=[]
  j=5000
  for i in range(200):#partitioning the book into 200 parts
    partition.append(raw[j:j+100])
    j+=100
  labeled=[(doc,label) for doc in partition]
  return labeled

Labeled_Book1=data_partition(book1,"J. Walker McSpadden")
Labeled_Book2=data_partition(book2,"Howard Pyle")
Labeled_Book3=data_partition(book3,"James Stephens")
Labeled_Book4=data_partition(book4,"William Hope Hodgson")
Labeled_Book5=data_partition(book5,"Thomas Malory")




In [176]:
Corpus=[]

for item in Labeled_Book1:
  Corpus.append(item)
for item in Labeled_Book2:
  Corpus.append(item)
for item in Labeled_Book3:
  Corpus.append(item)
for item in Labeled_Book4:
  Corpus.append(item)
for item in Labeled_Book5:
  Corpus.append(item)

random.shuffle(Corpus)
print(Corpus)

[(' for me, or she had never striven so to put me\r\nto shame before the stranger, and named me uncouth a', 'William Hope Hodgson'), ('s deer, and by the laws of King Harry your head remains forfeit.\r\nTalk not to me of pennies but get ', 'J. Walker McSpadden'), ('o watch, save that I was so\r\nheavy in the heart with loneliness and longing; for the company was\r\ngr', 'William Hope Hodgson'), (', and of a damosel which desired a knight to fight\r\n for a lady.\r\n\r\n CHAPTER III. How Beaumains desi', 'Thomas Malory'), ('s a beggar-man, a sorry looking fellow with leggings of\r\ndifferent colors, and brown scratched face ', 'J. Walker McSpadden'), ('se\r\nthe best hart among them to die."\r\n\r\n"Now done!" cried he who had spoken first.  "And here are t', 'Howard Pyle'), ('ervants asked what should be done with the foot-pads;\r\nseeing that they were now recovering. But, in', 'William Hope Hodgson'), ('t out to the men; for I heard their cries a good while after we had\r\ngone away.

In [177]:
df = pd.DataFrame(Corpus)
df.columns=['dict','label']
df['dict_clean']=df['dict'].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True))
df.head()

,dict,label,dict_clean
0,"for me, or she had never striven so to put me...",William Hope Hodgson,for me or she had never striven so to put me t...
1,"s deer, and by the laws of King Harry your hea...",J. Walker McSpadden,s deer and by the law of king harry your head ...
2,"o watch, save that I was so\r\nheavy in the he...",William Hope Hodgson,o watch save that i wa so heavy in the heart w...
3,", and of a damosel which desired a knight to f...",Thomas Malory,and of a damosel which desired a knight to fig...
4,"s a beggar-man, a sorry looking fellow with le...",J. Walker McSpadden,s a beggarman a sorry looking fellow with legg...


In [178]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['dict_clean'])
X_train_counts.shape


(1000, 3170)

In [179]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(1000, 3170)

In [181]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf,df['label'])